In [82]:
def recognize(string):  # функция, распознающая цепочки КС языка
    x=0  # счётчик для x
    skipped = False  # флаг того, скакой стороны цепочки текущие символы 'x'
    a=0  # счётчик для a
    b=0  # счётчик для b
    alphabet = ('x', 'a', 'b')  # алфавит
    for i in range(0, len(string)):
        if string[i] not in alphabet:
            return f"Ошибка: в позиции {i} неожидаемый символ {string[i]}!"
        if string[i] == 'x':
            if (string[i-1]=='a' or (b!=3*a/2)) and i!=0:
                print(i)
                print(a)
                print(b)
                print('in x')
                return f"Ошибка: в позиции {i} неожидаемый символ {string[i]}!"
            if not skipped:
                x = x + 1
            else:
                x = x - 1
        if string[i]=='a':
            if (string[i-1]=='b' and i!=0) or (string[i-1]=='x' and skipped):
                print('in a')
                return f"Ошибка: в позиции {i} неожидаемый символ {string[i]}!"
            a = a + 1
            skipped = True
        if string[i]=='b':
            if string[i-1]=='x' or a%2==1 or i==0:
                print('in b')
                return f"Ошибка: в позиции {i} неожидаемый символ {string[i]}!"
            b = b + 1
        if i==len(string)-1:
            if b>3*a/2:
                return f"Ошибка: в позиции {i} неожидаемый символ {string[int((a/2)*5)]}!"
            if b<3*a/2:
                 return f"Ошибка: в позиции {len(string)} неожидаемый символ конца цепочки!"
    if x > 0 and skipped:
        return f"Ошибка: в позиции {len(string)} ожидается символ x!"
    if x < 0 and skipped:
        j = string.find('a')
        return f"Ошибка: в позиции {j} неожидаемый символ {string[j]}!"
    return "Цепочка подходит"

In [142]:
class Automat():
    # таблца состояний (мертвое состояние всегда последнее)
    states = [
        ('a', 'b', 'x'),
        {'x':(2, ('',), 'm'), 'a': (3, ('',), ''), 'b': (9, ('',), '')},
        {'x':(2, ('m',), 'm'), 'a': (3, ('m',), ''), 'b': (9, ('m',), '')},
        {'x':(9, ('m',''), ''), 'a': (4, ('m','n',''), 'n'), 'b': (9, ('m','n',''), '')},
        {'x':(9, ('m','n',''), ''), 'a': (3, ('n',), ''), 'b': (5, ('n',), '')},
        {'x':(9, ('n',), ''), 'a': (9, ('n',), ''), 'b': (6, ('n',), '')},
        {'x':(9, ('n',), ''), 'a': (9, ('n',), ''), 'b': (7, ('n',), 'd')},
        {'x':(8, ('m',), 'd'), 'a': (9, ('m','n'), ''), 'b': (5, ('n',), '')},
        {'x':(9, ('m',), 'd'), 'a': (9, ('m',), ''), 'b': (9, ('m',), '')},
        {'x':(9, ('n','m',''), 'd'), 'a': (9, ('n','m',''), 'd'), 'b': (9, ('n','m',''), 'd')},  
    ]
    stack = ''  # начальный символ
    
    def __init__(self, final_states, current_state=1, states=None):  # конструктор
        self.final_states = final_states  # список финальных состояниц
        self.current_state = current_state  # текущее состояние
        if states is not None:
            self.set_states(states)
        
    def set_states(self, states):  # сеттер таблицы состояний
        self.states = states
        
    def add(self, string):  # добавить в конец стека символ
        self.stack = self.stack + string
    def pop(self):  # извлечь из стека последний символ
        self.stack = self.stack[0:-1]
    def check(self):  # посмотреть последниций символ в стеке
        if self.stack == '':
            return ''
        return self.stack[-1]
    
    def read(self, char):  # переход из одного состояний в другое основываясь на считанном символе и верхнем эементе стека
        if char not in self.states[0]:  # если символ не в алфавите переходим в мёртвое состояние
            self.current_state = self.states[-1][self.states[0][0]][0]
        state = self.states[self.current_state][char]
        if self.check() in state[1]:  # смотрим, что вверху стека
            if state[2]=='d':  # символ d означает удаление верхнего символа стека
                self.pop()
            else:  # иначе вносим символы в стек, основываясь на данных их таблицы состояний
                self.add(state[2])
            self.current_state = state[0]  # обновляем текущее состояние
        else:  # если нет совпадения верхнего символа стека с предполагаемыми на этом шаге 
            self.current_state = self.states[-1][self.states[0][0]][0]  # обновляем текущее состояние на мёртвое
        return self.current_state  # возвращаем состояние, в которое перешли
    
    def recognize(self, string):  # распознавание цепочки, допустимой автоматом
        for i in range(0,len(string)):  # проходим циклом по символам церпочки
            self.read(string[i])  # применыяем функцию перехода к каждому символу строки
            if self.current_state == len(self.states)-1:  # если перешли в мёртвое состояние
                print(f"Ошибка: в позиции {i} неожидаемый символ {string[i]}!")  # сообщение об ошибке
                self.stack = ''
                self.current_state = 1
                return False
        # если текущее состояние после работы автомата - одно из допустимых и стек пуст, то цепочка допустима
        if self.current_state in self.final_states and self.stack=='':
            self.stack = ''
            self.current_state = 1
            return True
        self.current_state = 1
        self.stack = ''
        print(f"Ошибка: в позиции {len(string)} не хватает символа!")
        return False

In [143]:
automat=Automat(final_states=(1,7,8))

In [144]:
automat.recognize('')

True

In [145]:
automat.recognize('xaabbbx')

True

In [146]:
automat.recognize('xaabbb')

Ошибка: в позиции 6 не хватает символа!


False

In [147]:
automat.recognize('aabbbx')

Ошибка: в позиции 5 неожидаемый символ x!


False

In [148]:
automat.recognize('xxaabbbx')

Ошибка: в позиции 8 не хватает символа!


False

In [149]:
automat.recognize('xaabbbxx')

Ошибка: в позиции 7 неожидаемый символ x!


False

In [150]:
automat.recognize('aabbb')

True

In [151]:
automat.recognize('aaabbb')

Ошибка: в позиции 3 неожидаемый символ b!


False

In [152]:
automat.recognize('aabbbb')

Ошибка: в позиции 5 неожидаемый символ b!


False

In [153]:
automat.recognize('xxxaabbbaaaxxx')

Ошибка: в позиции 8 неожидаемый символ a!


False